In [1]:
!pip install -q pinecone-client
!pip install -U -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00


In [2]:
!pip install -q jsonlines

In [3]:
from pinecone import Pinecone
from dotenv import load_dotenv
import os

load_dotenv(override=True)

PINECONE_API_KEY = os.getenv("pinecone_api_key")
PINECONE_INDEX = os.getenv("pinecone_index")

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX)

In [4]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.02694,
 'namespaces': {'': {'vector_count': 2694}},
 'total_vector_count': 2694}

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd drive/MyDrive

/content/drive/MyDrive


In [7]:
import json, jsonlines
import pandas as pd

def load_and_clean_json(file_path):
    try:
        with jsonlines.open(file_path, 'r') as reader:
            # Read each line separately
            lines = [line for line in reader]

            return lines

    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
file_path = 'DATA_final.json'
DATA = load_and_clean_json(file_path)
DATA

[{'amende': 'le dépassement non réglementaire ',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement illégal',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement horslaloi',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non autorisé',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non permis',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non encadré',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non conforme',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non contrôlé',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non surveillé',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non conventionnel',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non approuvé',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépa

In [8]:
# first we need the encodings
import torch
corpus_embeddings = torch.load("corpus_embeddings.pth")

In [9]:
corpus_embeddings.shape

torch.Size([2694, 768])

In [34]:
def batch_insert(batch_size=100):
  batch=0
  vectors=[]
  i=0
  for data, encoding in zip(DATA,corpus_embeddings) :
    # print(data, encoding)
    # break
    i+=1
    vector={"id":f'id{i}','values':'', "metadata":{"class_violation":''}}
    vector['metadata']['class_violation']=str(data['feature_class'])
    vector['values']=encoding
    vectors.append(vector)
    batch+=1
    if batch>=batch_size:
      index.upsert(vectors=vectors)
      vectors=[]
      batch=0
  # Insert any remaining vectors if the total is not a perfect multiple of batch_size
  if vectors:
      index.upsert(vectors=vectors)

In [35]:
batch_insert()

In [37]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.02694,
 'namespaces': {'': {'vector_count': 2694}},
 'total_vector_count': 2694}

In [ ]:
normal_values = [tensor.item() for tensor in corpus_embeddings[0]]
normal_values

In [21]:
result = index.query(vector=normal_values, top_k=1,include_metadata=True)
result

{'matches': [{'id': 'id1',
              'metadata': {'class_violation': "['400 DH', '500 DH', 4]"},
              'score': 1.0006597,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [24]:
result['matches'][0]['metadata']['class_violation']

"['400 DH', '500 DH', 4]"

In [25]:
from sentence_transformers import SentenceTransformer,util

model = SentenceTransformer('antoinelouis/biencoder-electra-base-french-mmarcoFR')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/227k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/731k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [34]:
def simulatechat(query):
  '''
  1. embed query
  2. reformat 
  3. pass to pinecone 
  4. get class
  5. custum sentence 
  '''
  embedding = model.encode(query)
  embedding =  [tensor.item() for tensor in embedding]
  result = index.query(vector=embedding, top_k=1,include_metadata=True)
  return result['matches'][0]['metadata']['class_violation']

In [35]:
query = "le nonrespect par le conducteur d'une signalisation d'arrêt par un panneau de stop"
classe = simulatechat(query)
print(classe)

['400 DH', '500 DH', 4]
